In [14]:
from ipywidgets import interact, IntSlider, FloatSlider, GridBox, Layout, Label, HTML
from mortgage import Mortgage, ureg, Q

In [15]:
def constant_overpayment_strategy(amount: Q):
    def overpayment(remaining: Mortgage) -> Q:
        return amount
    return overpayment

In [19]:
@interact
def summarize_scenarios(
        home_value : FloatSlider(min=3e5, max=1e7, description="Home value ($)"),
        down_payment_percentage : FloatSlider(min=0, max=45, description="Down payment (%)"),
        n_years : IntSlider(min=10, max=30, description="# of years"),
        n_discount_points : FloatSlider(min=-1, max=1, description="# discount points"),
        overpayment_amount : FloatSlider(min=0, max=2e4, description="Overpayment ($ / mo)")
    ):
    mortgage = Mortgage(
        home_value=Q(home_value, 'dollars'),
        loan_amount=Q(home_value, 'dollars') * (1 - Q(down_payment_percentage, 'percent')),
        duration=Q(n_years, 'years')
    )
    overpayment_strategy = constant_overpayment_strategy(Q(overpayment_amount, 'dollars / month'))
    total_cost, actual_duration = mortgage.total_cost(Q(n_discount_points, 'dp'), overpayment_strategy=overpayment_strategy)
    
    return GridBox(
        children=[
            Label("# of payments:"),
            Label(f"{mortgage.n_payments}"),

            Label(f"Effective rate:"),
            Label(f"{mortgage.effective_rate(Q(n_discount_points, 'dp'))}"),

            Label(f"Down payment:"),
            Label(f"{mortgage.down_payment}"),

            Label(f"Closing costs:"),
            Label(f"{mortgage.actual_closing_costs(Q(n_discount_points, 'dp'))}"),

            Label(f"Base payment:"),
            Label(f"{mortgage.base_payment(Q(n_discount_points, 'dp'))}"),

            HTML(f"<strong>Total cost:</strong>"),
            HTML(f"<strong>{total_cost}</strong>"),

            HTML(f"<strong>Actual duration:</strong>"),
            HTML(f"<strong>{actual_duration.to('months')}</strong>")
        ],
        
        layout=Layout(
            width='70%',
            grid_template_columns='auto auto'
        )
    )

interactive(children=(FloatSlider(value=300000.0, description='Home value ($)', max=10000000.0, min=300000.0),…